> <span style="font-size: 20px; color: red;">IMPORTANT</span>
>
> Не смейте запускать данное чудо где-либо еще, крмое кегли, если не хотите дополнительно устанавливать лишние для себя пакета. Поберегите свою рабочую машинку, спасибо!

# Оглавление

<details>
<summary>Развернуть Оглавление</summary>

1. Постановка задачи
2. Импортирование библиотек
3. EDA Анализ
4. Препроцессинг
5. Алгоритмы
    1. Линейная регрессия
    2. SVM
    3. Decision Tree
    4. Random Forest
    5. KNN
6. Подбор гиперпараметров
    1. CatBoost
    2. Random Forest
    3. Gradient Boosting
7. Реализация ансамбля через стэкинг моделей
8. SHAP Анализ
    1. Waterfall
    2. Bar chart
    3. Beeswarm
9. Экспортирование результатов

</details>

# 1. Постановка задачи

## Первая лабораторная работа по вводнуму курсу ML.
`Цель: предсказание стоимости авто.`
<details>
<summary>Описание данных</summary>

- Brand: Name of the car manufacturer
- Year: Year of manufacture or release
- Model: Name or code of the car model
- Car/Suv: Type of the car (car or suv)
- Title: Title or description of the car
- UsedOrNew: Condition of the car (used or new)
- Transmission: Type of transmission (manual or automatic)
- Engine: Engine capacity or power (in litres or kilowatts)
- DriveType: Type of drive (front-wheel, rear-wheel, or all-wheel)
- FuelType: Type of fuel (petrol, diesel, hybrid, or electric)
- FuelConsumption: Fuel consumption rate (in litres per 100 km)
- Kilometres: Distance travelled by the car (in kilometres)
- ColourExtInt: Colour of the car (exterior and interior)
- Location: Location of the car (city and state)
- CylindersinEngine: Number of cylinders in the engine
- BodyType: Shape or style of the car body (sedan, hatchback, coupe, etc.)
- Doors: Number of doors in the car
- Seats: Number of seats in the car
- Price: Price of the car (in dollars) **TARGET**

</details>
<details>
<summary>Требования</summary>

1. Сделать первичный анализ данных EDA
2. Использовать стандартные алгоритмы ML, Linear Regression, DT etc (не менее 3)
3. Использовать алгоритмы подбора гиперпараметров, GridSearch, RandomSearch etc. (не менее 1)
4. Использовать анасамбль с помощью бустинга или беггинга (не менее 1)
5. Реализовать из нескольких моделей анасамбль с помощью стекинга (не менее 1)
6. Сделать анализ параметров полученной модели ( анализ интерпретируемости) SHAP etc. (не менее 1 модели)

</details>

# 2. Импортирование библиотек

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from tqdm.notebook import tqdm

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    IsolationForest,
)
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, KFold

from catboost import CatBoostRegressor

import shap

import optuna

import logging

logger = logging.getLogger('sklearnex')
logger.setLevel(logging.WARNING)

sns.set(
    palette='tab10',
    font_scale=1.2,
    rc={
        'figure.figsize': (9, 7),
        'axes.titlepad': 24,
        'axes.labelpad': 20,
        'axes.titlesize': 22,
        'figure.titlesize': 22,
        'figure.subplot.hspace': 0.3,
        'figure.subplot.wspace': 0.25,
    },
)

RAND_STATE = 42
np.random.seed(RAND_STATE)

# 3. EDA Анализ

`Данные датасеты будут использоваться в качестве тренировочных и тестовых.`

In [ ]:
df_orig = pd.read_csv('./x_train.csv')
df = df_orig.copy()
test_submit = pd.read_csv('./x_test.csv')
df.head()

`Установим соответствие между типами признаков, наличием пропущенных значений и количеством уникальных значений. В дальнейшем, для того чтобы уменьшить количество рутинной работы и повысить эффективность, мы изменим формат датасета и проведем его первичный анализ.`

In [ ]:
df.info()

`Из всех признаков большая их часть является строковыми. Очевидно, что первый нумерующий столбец "Unnamed: 0" является индексирующим и его можно исключить из списка. На основе его отсутствия можно сделать вывод о наличии 18 признаков и 1 документально установленный таргет (Price). Есть несколько отличительных признаков, имеющих в себе пропуски значений (Location, BodyType, Doors, Seats). Price, Year - числовые признаки, собственно, как и год. Но в отличие от первых двух, последний является float`

In [ ]:
df.nunique()

In [ ]:
description = {
    'Brand': 'Name of the car manufacturer',
    'Year': 'Year of manufacture or release',
    'Model': 'Name or code of the car model',
    'Car/Suv': 'Type of the car (car or suv)',
    'Title': 'Title or description of the car',
    'UsedOrNew': 'Condition of the car (used or new)',
    'Transmission': 'Type of transmission (manual or automatic)',
    'Engine': 'Engine capacity or power (in litres or kilowatts)',
    'DriveType': 'Type of drive (front-wheel, rear-wheel, or all-wheel)',
    'FuelType': 'Type of fuel (petrol, diesel, hybrid, or electric)',
    'FuelConsumption': 'Fuel consumption rate (in litres per 100 km)',
    'Kilometres': 'Distance travelled by the car (in kilometres)',
    'ColourExtInt': 'Colour of the car (exterior and interior)',
    'Location': 'Location of the car (city and state)',
    'CylindersinEngine': 'Number of cylinders in the engine',
    'BodyType': 'Shape or style of the car body (sedan, hatchback, coupe, etc.)',
    'Doors': 'Number of doors in the car',
    'Seats': 'Number of seats in the car',
    'Price': 'Price of the car (in dollars)',
}

In [ ]:
profile = ProfileReport(
    df,
    minimal=True,
    variables={'descriptions': description},
    correlations={
        'auto': {'calculate': True},
        'phi_k': {'calculate': True},
        'cramers': {'calculate': True},
    },
    interactions={'targets': ['Price']},
    html={'style': {'theme': 'flatly'}},
    missing_diagrams={'bar': True, 'heatmap': True},
)
profile.to_file('cars_dataset_overview.html')

`Итог таков, что из всех 75-ти представленных брендов, верх одерживает Toyota с резульатом в 17% от общего числа. Учитывая корреляцию, можно выделить сильную зависимость цены, от бренды, что важно и логично (чем премиальнее авто, тем цена растет выше). Отсюда мы также можем видеть и корреляцию других признаков по отношению к бренду. Ведь на то он и бренд, чтобы придерживаться своих стандартов и выпускать автомобили согласно нему. Отсюда, кстати, вытекает еще один признак - BodyType, который тоже добавляем в корзину для обработки.
Касательно года можно отметить, что большая часть автомобилей была произведена в промежуток с 2013 по 2020 годы, и лишь некоторые экземпляры мелькают в датасете с датами за 90е годы. Как минимум в виду данных мельчайших отклонений не имеет смысла строить графики, иначе они поедут.
Новая машина или б/у такжа входит в список важных атрибутов, которые можно будет использовать при анализе.
По типу двигателя люди часто и выбирают автомобиль, стараюстя присмотреть помощнее. Для приличия возмьем и этот атрибут на обработку.
Остальные параметры, такие как, например, FuelConsumption, Kilometres, CylindersinEngine либо являются числовыми, либо могут быть таковыми после пары преобразований.
Остальная часть не имеет особого смысла по разным причинам: отсутствие данных, низкая актуальность для данной работы и т.д. По ходу будет понятно, что и где используется.`


# 4. Препроцессинг

In [ ]:
y = df.pop('Price')

In [ ]:
def clearData(df):
    '''
    Преобразуем километры, расход топлива и количество цилиндров в чисто числовые значения
    А в местах, где ничего нет, мы просто добавим нули, так будет удобнее.
    '''
    df['Kilometres'] = (
        df['Kilometres'].str.replace('- / -|-', '0', regex=True).astype(float)
    )
    df['FuelType'] = df['FuelType'].str.replace(
        '-', 'Other'
    )
    df['FuelConsumption'] = df['FuelConsumption'].str.replace('-', 'NaN')
    df['FuelConsumption'] = (
        df['FuelConsumption'].str.replace('L / 100 km', '').astype(float)
    )
    df['FuelConsumption'] = df['FuelConsumption'].fillna(
        df['FuelConsumption'].median()
    )
    df['CylindersinEngine'] = df['CylindersinEngine'].str.replace(
        '-', '0'
    )
    df['CylindersinEngine'] = (
        df['CylindersinEngine'].str.replace('cyl|L', '', regex=True).astype(float)
    )
    df['Seats'] = df['Seats'].str.replace('Seats', '').str.strip()
    df['Seats'] = df['Seats'].fillna(df['Seats'].median()).astype(int)
    df = df.drop(
        columns=[
            'Unnamed: 0',
            'Model',
            'Title',
            'ColourExtInt',
            'Location',
            'Car/Suv',
            'Engine',
            'Doors',
        ]
    )

    return df


def applyOneHot(train, test):
    '''
    Применяем однократное кодирование к категориальным объектам
    '''
    cols = [
        'Brand',
        'UsedOrNew',
        'DriveType',
        'FuelType',
        'BodyType',
        'Transmission',
        'CylindersinEngine',
    ]
    enc = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)

    train_onehotted = pd.DataFrame(enc.fit_transform(train[cols]), index=train.index)
    test_onehotted = pd.DataFrame(enc.transform(test[cols]), index=test.index)

    col_names = enc.get_feature_names_out(enc.feature_names_in_)
    train_onehotted.columns = col_names
    test_onehotted.columns = col_names

    train = pd.concat([train.drop(columns=cols), train_onehotted], axis=1)
    test = pd.concat([test.drop(columns=cols), test_onehotted], axis=1)

    return train, test


def applyScaling(train, test, scaler):
    '''
    Приименяем масштабирование значений ко всему набору данных. Будем считать, что все объекты уже числовые.
    '''
    train = pd.DataFrame(
        scaler.fit_transform(train), columns=train.columns, index=train.index
    )
    test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)
    return train, test


def preprocess(train, test, scaler):
    '''
    Преобразуем датасет так, чтобы с ним можно было работать
    '''
    train = clearData(train)
    test = clearData(test)
    train, test = applyOneHot(train, test)
    return train, test

In [ ]:
# Валидация данных
X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.3, random_state=RAND_STATE
)
X_train, X_test = preprocess(X_train, X_test, StandardScaler())

In [ ]:
# Полные данные для подготовки окончательной модели
df, test_submit = preprocess(df, test_submit, StandardScaler())
test_submit.head()

`Далее делаем проверку на error terms или на русском - выбросы`

In [ ]:
outliers_detector = IsolationForest(n_estimators=1000, random_state=RAND_STATE).fit(df)
outliers_analysis = pd.DataFrame(outliers_detector.predict(df)).replace(
    {-1: 'Anomaly', 1: 'Normal'}
)

pd.concat(
    [
        outliers_analysis.value_counts().rename('Count'),
        outliers_analysis.value_counts(normalize=True).rename('Proportion').round(2),
    ],
    axis=1,
)

In [ ]:
df_orig[outliers_analysis[0] == 'Anomaly']

`По результатам отработки алгоритм не нашел ничего серьезного. Хотя очень странно наблюдать в датасете автомобили с 12ю дверьми. Возможно, мы имеем дело с автобусом.`

`Далее будем отмерять важность отбираемых признаков. SelectKBest оценит взаимосвязь предикторов с целевой переменной, используя статистические тесты, позволяя отобрать соответственно заданное количество и долю наилучших по заданному критерию признаков`

In [ ]:
selector = SelectKBest(f_classif).fit(df, y)
k_features = 20
important_features = selector.scores_ != np.inf

results = pd.DataFrame(
    {
        'Score': selector.scores_[important_features][:k_features],
        'Feature': df.columns[important_features][:k_features],
    }
).sort_values(by='Score', ascending=False)

_, ax = plt.subplots(figsize=(12, 12))
p = sns.barplot(ax=ax, data=results, x='Score', y='Feature')
_ = p.set_title('Feature Importance')

`Как уже и оговаривалось ранее, бренд - есть цена, и он сильно определяет итоговый ценник.`

# 5. Алгоритмы

In [ ]:
def cross_validation(X: pd.DataFrame, y: pd.DataFrame, model, cv: int = 5, **kwargs):
    '''
    Выполняем перекрестную проверку и возвращаем среднее значение для тренинга и тестового RMSE
    '''
    kf = KFold(n_splits=cv, shuffle=True, random_state=RAND_STATE)
    train_rmse = test_rmse = 0
    for i, (train_idx, valid_idx) in enumerate(tqdm(kf.split(X, y), total=cv)):
        valid_data = X.iloc[valid_idx]
        valid_labels = y.iloc[valid_idx]
        train_data = X.iloc[train_idx]
        train_labels = y.iloc[train_idx]

        fitted_model = model(**kwargs).fit(train_data.values, train_labels)

        train_pred = fitted_model.predict(train_data.values)
        valid_pred = fitted_model.predict(valid_data.values)

        train_rmse_cv = round(
            mean_squared_error(train_labels, train_pred, squared=False)
        )
        test_rmse_cv = round(
            mean_squared_error(valid_labels, valid_pred, squared=False)
        )
        print(f'K-Fold #{i + 1}:')
        print(f'    - Тренинг RMSE: [ {train_rmse_cv} ]')
        print(f'    - Тест RMSE: [ {test_rmse_cv} ]\n')

        train_rmse += train_rmse_cv
        test_rmse += test_rmse_cv

    train_rmse = int(train_rmse / cv)
    test_rmse = int(test_rmse / cv)
    print(f'Среднее тренинговое  RMSE: [ {train_rmse} ] | Среднее тестовое RMSE: [] {test_rmse} ]')

    return train_rmse, test_rmse

## 5.1. Линейная регрессия

In [ ]:
%%time
linear_reg = LinearRegression
_ = cross_validation(df, y, linear_reg)

## 5.2. SVM

In [ ]:
%%time
svm = SVR
svm_train_rmse, svm_test_rmse = cross_validation(df, y, svm)

> Уже лучше. Машина стабильно обучилась, переобучения нет

## 5.3. Decision Tree (DT)

In [ ]:
%%time
dtr = DecisionTreeRegressor
dtr_train_rmse, dtr_test_rmse = cross_validation(df, y, dtr)

## 5.4. Random Forest (RF)

In [ ]:
%%time
rf = RandomForestRegressor
rf_train_rmse, rf_test_rmse = cross_validation(df, y, rf)

## 5.5. KNN

In [ ]:
%%time
knn = KNeighborsRegressor
knn_train_rmse, knn_test_rmse = cross_validation(df, y, knn)

### Итог

In [ ]:
train_rmse = ['Train', svm_train_rmse, dtr_train_rmse, rf_train_rmse, knn_train_rmse]
test_rmse = ['Test', svm_test_rmse, dtr_test_rmse, rf_test_rmse, knn_test_rmse]
algos = ['Type', 'SVM', 'Decision Tree', 'Random Forest', 'KNN']
metrics = pd.DataFrame(data=[train_rmse, test_rmse], columns=algos)
metrics[metrics.columns[1:]] = round(metrics[metrics.columns[1:]] / 1000, 1)

In [ ]:
metrics_to_plot = metrics.melt(id_vars='Type').rename(columns=str.title)
_, ax = plt.subplots(figsize=(16, 9))
p = sns.barplot(metrics_to_plot, ax=ax, x='Variable', y='Value', hue='Type')
p.set_title('RMSE по моделям')
p.set_xlabel('Алгоритм')
p.set_ylabel('Значение (x1000)')
p.bar_label(p.containers[0])
_ = p.bar_label(p.containers[1])

# 6. Подбор гиперпараметров

`Для данного шага воспользуемся optuna. Для тех, кто не понимает, почему лучше использовать ее, то прошу прочесть недавнюю статью, которые рассказывают причину данного выбора:`

[Ссылка](https://medium.com/@abdalrahman_shahrour/optuna-vs-gridsearch-57227556c450)

## 6.1. CatBoost

In [ ]:
def objective_cat(trial):
    catboost_params = {
        'iterations': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'depth': trial.suggest_int('depth', 1, 12),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),
        'random_state': RAND_STATE,
        'verbose': False,
    }

    _cat = CatBoostRegressor(**catboost_params)
    _cat.fit(X_train, y_train)

    preds = _cat.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return rmse

In [ ]:
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=60)

In [ ]:
# Catboost
print('Best hyperparameters:', study_cat.best_params)
print('Best RMSE:', study_cat.best_value)

## 6.2. Random Forest

In [ ]:
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 4, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 60),
        'random_state': RAND_STATE,
    }

    _rf = RandomForestRegressor(**params)
    _rf.fit(X_train.values, y_train)

    preds = _rf.predict(X_test.values)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return rmse

In [ ]:
study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=60)

In [ ]:
# Random Forest
print('Лучшие гиперпараметры:', study_rf.best_params)
print('Лучший RMSE:', study_rf.best_value)

## 6.3. Gradient Boosting

In [ ]:
def objective_gb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'subsample': trial.suggest_float('subsample', 0.5, 0.9, step=0.05),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'random_state': RAND_STATE,
    }

    _gb = GradientBoostingRegressor(**params)
    _gb.fit(X_train, y_train)

    preds = _gb.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)

    return rmse

In [ ]:
study_gb = optuna.create_study(direction='minimize')
study_gb.optimize(objective_gb, n_trials=60)

In [ ]:
# Gradient Boosting
print('Лучшие гиперпараметры:', study_gb.best_params)
print('Лучший RMSE:', study_gb.best_value)

# 7. Реализация ансамбля черзе стэкинг моделей

In [ ]:
p1 = study_cat.best_params
p1['iterations'] = 2000
p1['verbose'] = False
p1['random_state'] = RAND_STATE

p2 = study_rf.best_params
p2['random_state'] = RAND_STATE

p3 = study_gb.best_params
p3['random_state'] = RAND_STATE

cat = CatBoostRegressor(**p1).fit(df, y)
rf = RandomForestRegressor(**p2).fit(df, y)
gb = GradientBoostingRegressor(**p3).fit(df, y)

In [ ]:
last = pd.DataFrame()
last['rf'] = rf.predict(df)
last['cat'] = cat.predict(df)
last['gb'] = gb.predict(df)

last_test = pd.DataFrame()
last_test['rf'] = rf.predict(test_submit)
last_test['cat'] = cat.predict(test_submit)
last_test['gb'] = gb.predict(test_submit)

In [ ]:
model = Ridge().fit(last, y)
preds = model.predict(last_test)
preds[preds < 0] = 0

# 8. SHAP Анализ

In [ ]:
explainer = shap.Explainer(cat)
shap_values = explainer(test_submit)

## 8.1. Waterfall

In [ ]:
shap.plots.waterfall(shap_values[0])

`По результату первой проверки видим больше влияние со стороны старого доброего 2005-го`

## 8.2. Bar chart

`По данному критерию, по сути как и по SelectKBest, обределяем важность наших фичей`

In [ ]:
shap.plots.bar(shap_values)

`Несмотря на некоторое отклонение по результатам анализов, сделанных ранее, год и пробег все также составляют сильное влияние на дальнейший прогноз`

# 9. Экспортирование результатов

In [ ]:
with open('output.csv', 'w') as file:
    file.write('ID,Price\n')
    for i, val in enumerate(gb.predict(test_submit)):
        file.write(f'{i},{val}\n')